# 🌊 Reactive Streams & Async Patterns

**Phase 3: System Design - Event-Driven Architecture**

**Master event sourcing, CQRS, reactive streams, and asynchronous communication patterns**

---

In [ ]:
// Reactive Streams Overview
println("🌊 REACTIVE STREAMS & ASYNC PATTERNS - EVENT-DRIVEN SYSTEMS")
println()

println("🎯 Reactive Manifesto Principles:")
println("✓ Responsive: Systems respond in timely fashion")
println("✓ Resilient: Systems remain functional despite failures")
println("✓ Elastic: Systems handle varying workloads")
println("✓ Message-Driven: Async message passing for decoupling")
println()

println("🔧 Core Reactive Patterns:")
println("✓ Event Sourcing: State as series of events")
println("✓ CQRS: Separate read/write models")
println("✓ Reactive Streams: Non-blocking backpressure")
println("✓ Event Streaming: Continuous event processing")
println("✓ Lambda Architecture: Batch + speed layer integration")


## 📡 Event Sourcing Architecture

**Complete state reconstruction from domain events - immutable audit logging**

In [ ]:
// Event Sourcing Implementation
import cats.effect.{IO, Ref}
import cats.implicits._

// Domain Events
sealed trait DomainEvent {
  def aggregateId: String
  def sequenceNumber: Long
  def timestamp: java.time.Instant
}

case class AccountCreated(
  aggregateId: String,
  ownerId: String,
  currency: String,
  sequenceNumber: Long = 1L,
  timestamp: java.time.Instant = java.time.Instant.now()
) extends DomainEvent

case class FundsDeposited(
  aggregateId: String,
  amount: BigDecimal,
  reference: String,
  sequenceNumber: Long,
  timestamp: java.time.Instant = java.time.Instant.now()
) extends DomainEvent

case class FundsWithdrawn(
  aggregateId: String,
  amount: BigDecimal,
  reference: String,
  sequenceNumber: Long,
  timestamp: java.time.Instant = java.time.Instant.now()
) extends DomainEvent

// Current State (Projection)
case class BankAccount(
  id: String,
  ownerId: String,
  balance: BigDecimal,
  currency: String,
  version: Long = 0L
)

// Event Store
trait EventStore[F[_]] {
  def saveEvents(events: List[DomainEvent]): F[Unit]
  def getEvents(aggregateId: String): F[List[DomainEvent]]
  def getEventsSince(aggregateId: String, fromSequence: Long): F[List[DomainEvent]]
}

// In-memory event store implementation
class InMemoryEventStore[F[_]: Async] extends EventStore[F] {
  private val events = Ref.unsafe[F, Map[String, List[DomainEvent]]](Map.empty)
  
  def saveEvents(events: List[DomainEvent]): F[Unit] = {
    events.groupBy(_.aggregateId).toList.traverse { case (aggId, newEvents) =>
      events.modify { current =>
        val existing = current.getOrElse(aggId, Nil)
        val updated = existing ++ newEvents
        (current + (aggId -> updated), ())
      }
    }.void
  }
  
  def getEvents(aggregateId: String): F[List[DomainEvent]] =
    events.get.map(_.getOrElse(aggregateId, Nil))
    
  def getEventsSince(aggregateId: String, fromSequence: Long): F[List[DomainEvent]] =
    getEvents(aggregateId).map(_.filter(_.sequenceNumber > fromSequence))
}

println("📡 Event Sourcing Implemented")
println("• Immutable event storage with replay capability")
println("• Domain events representing state changes")
println("• Sequence numbering for ordering guarantees")
println("• Audit trail and debugging capabilities")
println()


## 🔄 CQRS Architecture Pattern

**Separate Command and Query responsibility for optimized read/write patterns**

In [ ]:
// CQRS Implementation - Separate Read/Write Models

// Commands (Write Operations)
sealed trait AccountCommand {
  def accountId: String
}

case class OpenAccount(accountId: String, ownerId: String, currency: String) extends AccountCommand
case class DepositFunds(accountId: String, amount: BigDecimal) extends AccountCommand
case class WithdrawFunds(accountId: String, amount: BigDecimal) extends AccountCommand
case class CloseAccount(accountId: String) extends AccountCommand

// Queries (Read Operations)
sealed trait AccountQuery {
  def accountId: String
}

case class GetAccountBalance(accountId: String) extends AccountQuery
case class GetAccountHistory(accountId: String, since: java.time.Instant) extends AccountQuery
case class GetAccountSummary(accountId: String) extends AccountQuery

// Read Models (Projections)
case class AccountSummary(
  accountId: String,
  balance: BigDecimal,
  currency: String,
  ownerName: String,
  lastActivity: java.time.Instant
)

case class AccountActivity(
  accountId: String,
  eventType: String,
  amount: Option[BigDecimal],
  timestamp: java.time.Instant,
  reference: Option[String]
)

// CQRS System
trait CommandHandler[F[_]] {
  def handle(command: AccountCommand): F[Either[DomainError, List[DomainEvent]]]
}

trait QueryHandler[F[_]] {
  def handle(query: AccountQuery): F[Either[DomainError, Any]]
}

class CQRSBankAccountSystem[F[_]: Async](
  commandHandler: CommandHandler[F],
  queryHandler: QueryHandler[F],
  eventBus: EventBus[F]
) {

  def processCommand(command: AccountCommand): F[Either[DomainError, Unit]] =
    for {
      result <- commandHandler.handle(command)
      _ <- result.traverse(events => eventBus.publish(events))
    } yield result.map(_ => ())

  def executeQuery[A](query: AccountQuery): F[Either[DomainError, A]] =
    queryHandler.handle(query).map(_.map(_.asInstanceOf[A]))

  // Optimized read methods
  def getAccountBalance(accountId: String): F[Either[DomainError, BigDecimal]] =
    executeQuery[BigDecimal](GetAccountBalance(accountId))

  def getAccountSummary(accountId: String): F[Either[DomainError, AccountSummary]] =
    executeQuery[AccountSummary](GetAccountSummary(accountId))

  // Eventual consistency demonstration
  def getAccountHistory(accountId: String): F[Either[DomainError, List[AccountActivity]]] =
    executeQuery[List[AccountActivity]](
      GetAccountHistory(accountId, java.time.Instant.now().minusSeconds(300))
    )
}

sealed trait DomainError
case class InsufficientFunds(accountId: String, requested: BigDecimal, available: BigDecimal) extends DomainError
case class AccountNotFound(accountId: String) extends DomainError
case class InvalidAmount(amount: BigDecimal) extends DomainError

println("🔄 CQRS Pattern Implemented")
println("• Separate read/write models for optimization")
println("• Commands for state-changing operations")
println("• Queries for efficient data retrieval")
println("• Event-driven communication between sides")
println()


## 🌊 Reactive Streams with Backpressure

**Non-blocking, asynchronous stream processing with demand signaling**

In [ ]:
// Reactive Streams with Scala's fs2
import fs2.{Stream, Pipe, Pull}
import cats.effect.IO
import scala.concurrent.duration._

// Reactive stream processing pipeline
case class DataBatch(
  id: String,
  data: List[String],
  timestamp: java.time.Instant = java.time.Instant.now()
)

class ReactiveDataProcessor[F[_]: Async: Timer] {

  // Producer: Simulates data source with varying rates
  def dataSource: Stream[F, DataBatch] = {
    Stream.unfoldEval(0) { counter =>
      val batchSize = scala.util.Random.nextInt(100) + 50 // 50-150 items
      val data = (1 to batchSize).map(i => s"item_${counter}_$i").toList
      val batch = DataBatch(s"batch_$counter", data)
      
      for {
        _ <- Timer[F].sleep((scala.util.Random.nextInt(500) + 100).millis) // Random delay
      } yield Some((batch, counter + 1))
    }.take(20) // Finite stream
  }

  // Processing pipeline with backpressure
  def processingPipeline(input: Stream[F, DataBatch]): Stream[F, String] = {
    input
      . evalTap(batch => Async[F].delay(println(s"📦 Received batch: ${batch.id}")))
      . map(processBatch)                          // Transform
      . through(throttlingPipe)                   // Rate limiting
      . through(bufferWithBackpressure)          // Buffering
      . through(parallelProcessing(4))           // Parallel processing
      . through(errorRecovery)                    // Resilience
      . evalTap(result => Async[F].delay(println(s"✅ Processed: $result")))
  }

  private def processBatch(batch: DataBatch): List[String] = {
    batch.data.map(item => s"processed_$item")
  }

  // Throttling with backpressure
  private def throttlingPipe: Pipe[F, List[String], List[String]] = {
    _.groupWithin(1000, 2.seconds)  // Max 1000 items or 2 seconds
      .map(_.flatten.take(1000))    // Limit batch size
  }

  // Buffer with backpressure signaling
  private def bufferWithBackpressure: Pipe[F, List[String], List[String]] = {
    _.bufferSliding(3)  // Buffer up to 3 elements, drop oldest when full
  }

  // Parallel processing with bounded parallelism
  private def parallelProcessing(parallelism: Int): Pipe[F, List[String], String] = {
    _.mapAsync(parallelism) { batch =>
      Async[F].delay {
        Thread.sleep(100) // Simulate processing time
        s"BATCH_SIZE_${batch.size}"
      }
    }
  }

  // Error recovery with fallback
  private def errorRecovery: Pipe[F, String, String] = {
    _.handleErrorWith { error =>
      Stream.eval(Async[F].delay(println(s"❌ Processing error: $error"))) *>
      Stream.emit("ERROR_RECOVERED")
    }
  }

  // Complete reactive pipeline
  def runPipeline(): Stream[F, Unit] = {
    processingPipeline(dataSource)
      .drain // Convert to Unit stream
      .onComplete(Stream.eval(Async[F].delay(println("🎯 Pipeline completed successfully"))))
  }
}

println("🌊 Reactive Streams with Backpressure Implemented")
println("• Non-blocking stream processing")
println("• Automatic backpressure handling")
println("• Throttling and rate limiting")
println("• Parallel processing with bounded concurrency")
println("• Error recovery and resilience")


## 📡 Event Streaming with Kafka Patterns

**Distributed event streaming and real-time data processing**

In [ ]:
// Event Streaming Architecture
println("📡 EVENT STREAMING & REAL-TIME PROCESSING")
println()

// Stream processing patterns
println("🔥 Stream Processing Patterns:")
println()

println("1. 📊 Windowing Operations")
println("   ✓ Tumbling windows: Fixed-size, non-overlapping")
println("   ✓ Sliding windows: Overlapping with fixed advances")
println("   ✓ Session windows: Based on activity gaps")
println("   ✓ Time-based windows: Calendar-aligned intervals")
println()

println("2. 🔗 Stream Joins")
println("   ✓ Inner joins: Correlated events only")
println("   ✓ Left joins: All left events with optional right")
println("   ✓ Full outer joins: All events from both streams")
println("   ✓ Interval joins: Events within time windows")
println()

println("3. 🔄 Stateful Operations")
println("   ✓ Stateful aggregations with fault tolerance")
println("   ✓ Checkpointing for recoverable state")
println("   ✓ State stores with TTL expiration")
println("   ✓ Interactive queries on stream state")
println()

println("4. 📈 Event Enrichment")
println("   ✓ Lookups in external data sources")
println("   ✓ Joining with reference data")
println("   ✓ Geolocation and entity resolution")
println("   ✓ Real-time feature engineering")
println()

println("🏗️ Lambda Architecture Integration:")
println("• Batch layer: Historical data processing")
println("• Speed layer: Real-time event processing")
println("• Serving layer: Unified data access")
println("• Kappa architecture: Stream-first approach")
println()

println("⚡ Reactive System Guarantees:")
println("• At-least-once processing")
println("• Exactly-once semantics")
println("• Event time vs. processing time")
println("• Out-of-order event handling")
println("• Watermarking for completion")

println("\nThe key to reactive streams: handle data flow, not just individual elements!")
